In [16]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/NLP/project


In [2]:
%pip install transformers

In [3]:
!nvidia-smi

Tue Oct 26 01:58:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

from gpt2 import train, generate, save_model, load_model
from analysis import loss_plot, perplexity_plot
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, DataCollatorForLanguageModeling, LineByLineTextDataset
from transformers import AdamW 

from tqdm import tqdm

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = torch.device('cuda')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

# Training

## Elon Musk

In [10]:
!python run_language_modeling.py --output_dir=models/elonmusk --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/elonmusk/train.txt --eval_data_file=data/elonmusk/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:47:04 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:47:04 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Andrej Karpathy

In [8]:
!python run_language_modeling.py --output_dir=models/karpathy --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/karpathy/train.txt --eval_data_file=data/karpathy/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:16:07 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:16:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Barack Obama

In [9]:
!python run_language_modeling.py --output_dir=models/BarackObama --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/BarackObama/train.txt --eval_data_file=data/BarackObama/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=3

10/26/2021 02:26:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:26:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

# Generation

## Elon Musk

In [17]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:28 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Am excited about the Model 3 unveil and road trip this weekend. There are some huge changes coming to the design as well, so this one is easy & fast. 
=== GENERATED SEQUENCE 2 ===
To be clear, I am against artificial intelligence. AI should not be in charge. I want it to be. 
=== GENERATED SEQUENCE 3 ===
The last remaining soul mate of the late King Arthur was actually Dragan Motavalli. He was one of the greats. 
=== GENERATED SEQUENCE 4 ===
My kids love  @SpaceX  and  @NASA. 
=== GENERATED SEQUENCE 5 ===
It's not just me: I have several cats. I also own a chicken

## Andrej Karpathy

In [18]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/karpathy" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:40 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
"How to Start a Startup" from  @OpenAI  great resource, tips & tricks! 
=== GENERATED SEQUENCE 2 ===
A new Deep Learning tutorial for beginners:  the DeepMind GTC231n video   (w)’s A New Challenge for Computer Visionists 
=== GENERATED SEQUENCE 3 ===
The Deep Web is a Hotbed of Violence and Corruption  soooo excited! 
=== GENERATED SEQUENCE 4 ===
#cvpr2016  a week of hacking on the ConvNetCVPR code. Looks like I'm on a roll with the demos 
=== GENERATED SEQUENCE 5 ===
It's quite amusing to watch what is happening behind closed doors with no public hearings or publ

## Barack Obama

In [19]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/BarackObama" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:52 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/BarackObama', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
“They’re going to try to scare us off education entirely, because no one wants to go to college.” 
=== GENERATED SEQUENCE 2 ===
"That said, I know that people can still make their voices heard in Washington. I know it's not easy to do that." —President Obama 
=== GENERATED SEQUENCE 3 ===
The Senate must make a timely vote on an American Jobs Act now. The Jobs Act needs to be included in any deal. 
=== GENERATED SEQUENCE 4 ===
There's still more work to do, but this is progress. 
=== GENERATED SEQUENCE 5 ===
It's up to us to work with leaders in both parties to 